<a href="https://colab.research.google.com/github/whokilleddb/Creep-Detector/blob/main/CreepDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries 
import pandas as pd
import re
import xml.etree.ElementTree as ET
import os
from math import radians, cos, sin, asin, sqrt
from IPython.display import display
import folium
from google.colab import drive
from google.colab.output import eval_js

In [ ]:
# mount drive
drive.mount('/content/gdrive')

In [ ]:
# Read files
tree = ET.parse('./gdrive/MyDrive/Creep-Detector/session_data/Kismet-20191011-18-31-55-1.netxml')
gps_tree = ET.parse('./gdrive/MyDrive/Creep-Detector/session_data/Kismet-20191011-18-31-55-1.gpsxml')

In [ ]:
# Get Root Node
root = tree.getroot()
gps_root = gps_tree.getroot()

In [ ]:
# Read Mac List and define function to return Vendor
file = open('./gdrive/MyDrive/Creep-Detector/mac-vendors.txt', 'r', encoding="utf8")
mac_dict={}
for line in file:
  mac_dict[line.split('\t')[0]]=line.split('\t')[1].replace('\n','')

def getDeviceType(bssid): 
  return mac_dict.get(bssid[:8].upper())

In [ ]:
# haversine formula GPS coordinate distance
def haversine(lon1, lat1, lon2, lat2): 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r


In [ ]:
def appendArray(net, child): 
    if (child.tag == net) :
        bssid = ""
        essid = ""
        encryption = [] 
        time = []
        gps = [] 
        if net == "wireless-network":
          scan = 'BSSID'
        else:
          scan = 'client-mac'

        for bssid_tag in child.findall(scan): 
          bssid = bssid_tag.text 
        for gps_tag in child.findall("gps-info"):
            gps = [[float(gps_tag[0].text), float(gps_tag[1].text)],[float(gps_tag[4].text), float(gps_tag[5].text)]]
        
        time = [child.attrib['first-time'], child.attrib['last-time']]
        
        for ssid_tag in child.findall('SSID'):
            for essid_tag in ssid_tag.findall('essid'): 
              essid= essid_tag.text
            for info_tag in ssid_tag:
              if info_tag.tag == "encryption":
                encryption.append(info_tag.text)

        # add individual device entry
        StationList.append([bssid,gps,time,getDeviceType(bssid),encryption,essid])  

In [ ]:
# create array of all devices found
StationList = [] # MAC, GPS[[min],[max]], time[[first seen],[last seen]], device type, encryption, ESSID
for child in root:
    appendArray("wireless-network", child)
    for subchild in child:
        appendArray("wireless-client", subchild) # client under network tag

In [ ]:
# creep list with threshold of .20 km
threshold = .20

creeps=[]
for index in range(0,len(StationList)):
  dist = haversine(StationList[index][1][0][0],StationList[index][1][0][1],StationList[index][1][1][0],StationList[index][1][1][1])
  if dist > threshold :
    creeps.append(StationList[index])


In [ ]:
# remove duplicates
unique = []
duplicate = []

for creep, i in zip(creeps, range(len(creeps))) :
    if creep[0] not in unique: 
      unique.append(creep[0])
    else: 
      duplicate.append(i)

for index in sorted(duplicate, reverse=True): 
  del creeps[index]  

In [ ]:
# check source MAC
for child in creeps : 
  for gpss in gps_root:
    if (('source' in gpss.attrib) and gpss.attrib['source']==child[0]):
      child[1].append([float(gpss.attrib["lat"]),float(gpss.attrib["lon"])])

pd.DataFrame(creeps, columns=["BSSID", "GPS","Dates","Manufacturer","Encryption","ESSID"])

In [ ]:
# gather all gps points
points=[]
for child in gps_root: 
  if child.tag == "gps-point":
    points.append([float(child.attrib['lat']),float(child.attrib['lon'])])

coordinates = list(set(map(tuple,points)))

In [ ]:
# function to find closest point to current index and sort a path
def findClose(loop) :
    for iter in range(0,loop) :
        for coord in range(0, len(coordinates)):
            low = 10000000000000000
            for nextc in range(coord+1, len(coordinates)):
              distance=haversine(coordinates[coord][0],coordinates[coord][1],coordinates[nextc][0],coordinates[nextc][1]) 
              if (distance < low): # closest 
                    low = distance
                    index = nextc
            coordinates.insert(coord+1, coordinates.pop(index))
        coordinates.reverse() # reverse list since it always almost ends up at the correct finishing point
  

In [ ]:
def findFar(loop):
  large = 0
  for iter in range(0,loop) :
    for i in range(0, len(coordinates)-1):
        if haversine(coordinates[i][0],coordinates[i][1],coordinates[i+1][0],coordinates[i+1][1]) > large:
            large = haversine(coordinates[i][0],coordinates[i][1],coordinates[i+1][0],coordinates[i+1][1])
            index = i
    coordinates.insert(0, coordinates.pop(index))

In [ ]:
def get_center():
  max_lat = -91.0
  max_long = -181.0
  min_lat = 91.0
  min_long = 181.0
  for location in coordinates:
    max_lat = location[0] if location[0] >= max_lat else max_lat 
    max_long = location[1] if location[1] >= max_long else max_long
    min_lat = location[0] if location[0] <= min_lat else min_lat 
    min_long = location[1] if location[1] <= min_long else min_long
  return [(min_lat+max_lat)/2.0,(min_long+max_long)/2.0]

In [ ]:
# map reference frame, plot path
findClose(2) # sort twice
#findFar(2) #  Adjust Accordingly
LDN_COORDINATES = get_center()
myMap = folium.Map(location=LDN_COORDINATES, zoom_start=16, tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr = 'Esri')
folium.PolyLine(coordinates,line_opacity = 0.5, weight = 4).add_to(myMap)

In [ ]:
colors = [ 'red', 'blue', 'green', 'orange', 'purple', 'beige', 'gray', 'pink', 'black', 'lightgreen', 'darkblue', 'lightblue', 'lightred', 'darkpurple', 'darkred', 'cadetblue', 'lightgray', 'darkgreen' ]

# hacky solution to plot creeps
for x in range(0, len(creeps)): 
    globals()['creep%s' % x] = folium.FeatureGroup(name = 'Creep '+ str(x))
    for creep_loc in creeps[x][1]: 
      bssid = creeps[x][0]
      first_seen = creeps[x][2][0]
      last_seen = creeps[x][2][1]
      macvendor = f"MAC Vendor: {creeps[x][3]}" if creeps[x][3] != None and len(creeps[x][3]) != 0  else ''
      ssid = creeps[x][5] if creeps[x][5] != None and len(creeps[x][5]) != 0 else '--'
      if ssid != '--':
        folium.Marker(location=creep_loc, tooltip=f"SSID: {ssid}", popup=f"SSID: {ssid}\nBSSID: {bssid}\nFirst Seen: {first_seen}\nLast Seen: {last_seen}\n{macvendor}", icon=folium.Icon(color=colors[x % len(colors)], icon='user-secret', prefix='fa')).add_to(globals()['creep%s' % x])  
      else:
        folium.Marker(location=creep_loc, tooltip=f"BSSID: {bssid}", popup=f"BSSID: {bssid}\nFirst Seen: {first_seen}\nLast Seen: {last_seen}\n{macvendor}", icon=folium.Icon(color=colors[x % len(colors)], icon='user-secret', prefix='fa')).add_to(globals()['creep%s' % x])  
    globals()['creep%s' % x].add_to(myMap)

# add toggle for creeps
folium.LayerControl().add_to(myMap)
if os.path.exists('./gdrive/MyDrive/Creep Detector/index.html'):
  os.remove('./gdrive/MyDrive/Creep-Detector/index.html')
myMap.save('./gdrive/MyDrive/Creep-Detector/index.html')
display(myMap)

In [ ]:
# Create A Globally Accessible URL
print(eval_js("google.colab.kernel.proxyPort(8000)"))

In [ ]:
# Host A Python Server
!python3 -m http.server 8000 --directory ./gdrive/MyDrive/Creep-Detector/